In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
import sys 
sys.path.append("..")
from utils.database_managers import QDrantDBManager
from utils.embedding import EmbeddingFunction
from utils.text_processing import TextSplitter
from utils.language_models import LangChainAI
from langchain.document_loaders.csv_loader import CSVLoader

In [2]:
filename="../data/retail-info-test.csv"

In [3]:
text_splitter = TextSplitter(
    chunk_size=2000, 
    chunk_overlap=20
)
embedding = EmbeddingFunction('openAI').embedder
QDRANT_URL=os.getenv('QDRANT_URL')
COLLECTION_NAME="csv-openAI-embed"

qdrantClient = QDrantDBManager(
    url=QDRANT_URL,
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=1536,
    embedding=embedding,
    record_manager_url="sqlite:///record_manager_cache.sql"
)
retriever = qdrantClient.vector_store.as_retriever()

c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [4]:
loader = CSVLoader(file_path=filename)
docs = loader.load()

In [5]:
langchain_client = LangChainAI()
docs = langchain_client.add_metadata(docs, metadata={"source": "csv", "product": "monitor", "priority": "1"})

c:\Users\ELAFACRB1\Codice\GitHub\riassume\riassume\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# splitted_docs = text_splitter.fixed_split(docs)

In [7]:
qdrantClient.index_documents(docs)